<a href="https://colab.research.google.com/github/sljm12/machine_learning_notebooks/blob/master/Newsheadlines_Simple_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sample code for using Simple Transformers to train a News Headlines classifier.

The dataset is from https://www.kaggle.com/rmisra/news-category-dataset

Check GPU specs

In [1]:
!nvidia-smi

Fri Oct  2 06:24:26 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip -q install simpletransformers

     |████████████████████████████████| 215kB 5.4MB/s 
     |████████████████████████████████| 71kB 7.0MB/s 
     |████████████████████████████████| 317kB 17.4MB/s 
     |████████████████████████████████| 1.1MB 25.6MB/s 
     |████████████████████████████████| 7.2MB 40.0MB/s 
     |████████████████████████████████| 3.0MB 42.0MB/s 
     |████████████████████████████████| 1.7MB 33.9MB/s 
     |████████████████████████████████| 890kB 43.8MB/s 
     |████████████████████████████████| 1.1MB 41.4MB/s 
     |████████████████████████████████| 6.7MB 29.5MB/s 
     |████████████████████████████████| 133kB 46.3MB/s 
     |████████████████████████████████| 112kB 45.1MB/s 
     |████████████████████████████████| 4.4MB 35.6MB/s 
     |████████████████████████████████| 102kB 10.3MB/s 
     |████████████████████████████████| 102kB 11.2MB/s 
     |████████████████████████████████| 122kB 41.4MB/s 
     |████████████████████████████████| 163kB 44.7MB/s 
     |████████████████████████████████| 71kB 9.2MB/

In [3]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import numpy as np
import tensorflow as tf
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)



wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


#Training

In [4]:
!wget -qq https://www.dropbox.com/s/duoi46s4db28xac/news-category-dataset.zip
!unzip news-category-dataset.zip

Archive:  news-category-dataset.zip
  inflating: News_Category_Dataset_v2.json  


In [5]:
import json
from sklearn.utils import shuffle
data=[]
with open("/content/News_Category_Dataset_v2.json") as f:
  lines = f.readlines()
  for l in lines:
    j=json.loads(l)
    data.append(j)
data_size=len(data)
print("Data Size",data_size)
data=shuffle(data) #Shuffling of the data is impt without it the scores are very bad.

Data Size 200853


In [6]:
train_num = int(data_size*0.6)
val_num = int(data_size * 0.2)
test_num = int(data_size * 0.2)

In [7]:
df=pd.DataFrame(data=data)
num_categories=len(df.groupby("category").count())
print("Number of categories", num_categories)

Number of categories 41


In [8]:
df.head()

,category,headline,authors,link,short_description,date
0,FIFTY,Death Comes For Fluffy The Cat,"Roz Warren, ContributorAuthor of OUR BODIES, O...",https://www.huffingtonpost.com/entry/death-com...,,2016-04-24
1,STYLE & BEAUTY,Marion Cotillard Gets Handsy On The Cover Of W...,Rebecca Adams,https://www.huffingtonpost.com/entry/marion-co...,Also: is that an Amelie-esque hairdo we spy?,2012-11-18
2,ENTERTAINMENT,Viewers Threaten Bloody Murder If Dr. Bailey D...,Ron Dicker,https://www.huffingtonpost.com/entry/dr-bailey...,A preview showing the beloved character having...,2018-01-26
3,GOOD NEWS,Wheelchair Turns Boy With Cerebral Palsy Into ...,Elyse Wanshel,https://www.huffingtonpost.com/entry/wheelchai...,Let the good times roll.,2016-02-12
4,WELLNESS,"Blue Jasmine, the Samson Complex, and the Trau...","Naomi Kehati Bronner, Contributor\nClinical ps...",https://www.huffingtonpost.com/entry/blue-jasm...,"Blue Jasmine, Woody Allen's recent movie, is a...",2013-10-16


In [9]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df["cat_codes"]=le.fit_transform(df["category"])

In [10]:
df.head()

,category,headline,authors,link,short_description,date,cat_codes
0,FIFTY,Death Comes For Fluffy The Cat,"Roz Warren, ContributorAuthor of OUR BODIES, O...",https://www.huffingtonpost.com/entry/death-com...,,2016-04-24,12
1,STYLE & BEAUTY,Marion Cotillard Gets Handsy On The Cover Of W...,Rebecca Adams,https://www.huffingtonpost.com/entry/marion-co...,Also: is that an Amelie-esque hairdo we spy?,2012-11-18,30
2,ENTERTAINMENT,Viewers Threaten Bloody Murder If Dr. Bailey D...,Ron Dicker,https://www.huffingtonpost.com/entry/dr-bailey...,A preview showing the beloved character having...,2018-01-26,10
3,GOOD NEWS,Wheelchair Turns Boy With Cerebral Palsy Into ...,Elyse Wanshel,https://www.huffingtonpost.com/entry/wheelchai...,Let the good times roll.,2016-02-12,14
4,WELLNESS,"Blue Jasmine, the Samson Complex, and the Trau...","Naomi Kehati Bronner, Contributor\nClinical ps...",https://www.huffingtonpost.com/entry/blue-jasm...,"Blue Jasmine, Woody Allen's recent movie, is a...",2013-10-16,37


In [11]:
cat_data=df.groupby(["category"]).count()
labels = cat_data.index
labels 
# The LabelEncoder is following the order in labels

Index(['ARTS', 'ARTS & CULTURE', 'BLACK VOICES', 'BUSINESS', 'COLLEGE',
       'COMEDY', 'CRIME', 'CULTURE & ARTS', 'DIVORCE', 'EDUCATION',
       'ENTERTAINMENT', 'ENVIRONMENT', 'FIFTY', 'FOOD & DRINK', 'GOOD NEWS',
       'GREEN', 'HEALTHY LIVING', 'HOME & LIVING', 'IMPACT', 'LATINO VOICES',
       'MEDIA', 'MONEY', 'PARENTING', 'PARENTS', 'POLITICS', 'QUEER VOICES',
       'RELIGION', 'SCIENCE', 'SPORTS', 'STYLE', 'STYLE & BEAUTY', 'TASTE',
       'TECH', 'THE WORLDPOST', 'TRAVEL', 'WEDDINGS', 'WEIRD NEWS', 'WELLNESS',
       'WOMEN', 'WORLD NEWS', 'WORLDPOST'],
      dtype='object', name='category')

In [12]:
#One hot encoding
encodes=tf.one_hot(df["cat_codes"],num_categories)

In [13]:
#Sanity check on the labels and the one hot encoding
arg_max_label ={}
for l in labels:
  a= df[df["category"] == l]
  row = a.index[0]
  arg_max= np.argmax(encodes[row])
  arg_max_label[arg_max]=l
  print(l,arg_max)

ARTS 0
ARTS & CULTURE 1
BLACK VOICES 2
BUSINESS 3
COLLEGE 4
COMEDY 5
CRIME 6
CULTURE & ARTS 7
DIVORCE 8
EDUCATION 9
ENTERTAINMENT 10
ENVIRONMENT 11
FIFTY 12
FOOD & DRINK 13
GOOD NEWS 14
GREEN 15
HEALTHY LIVING 16
HOME & LIVING 17
IMPACT 18
LATINO VOICES 19
MEDIA 20
MONEY 21
PARENTING 22
PARENTS 23
POLITICS 24
QUEER VOICES 25
RELIGION 26
SCIENCE 27
SPORTS 28
STYLE 29
STYLE & BEAUTY 30
TASTE 31
TECH 32
THE WORLDPOST 33
TRAVEL 34
WEDDINGS 35
WEIRD NEWS 36
WELLNESS 37
WOMEN 38
WORLD NEWS 39
WORLDPOST 40


In [14]:
train_df = df[:train_num]
val_df = df[train_num:train_num+val_num]
test_df = df[train_num+val_num:]

In [15]:
print(len(train_df))
print(len(val_df))
print(len(test_df))

120511
40170
40172


In [16]:
train = train_df[['headline','cat_codes']]
test = test_df[['headline','cat_codes']]

In [ ]:
num_labels = len(train['cat_codes'].unique())

In [ ]:
# Create a ClassificationModel
model = ClassificationModel('bert', 'bert-base-cased', num_labels=num_labels, use_cuda=True, 
                            args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs':10, 'train_batch_size':128})
# You can set class weights by using the optional weight argument

# Train the model
model.train_model(train)



Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to outputs/.


In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_set)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:810: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_model:{'mcc': -0.010226134499304049, 'eval_loss': 9.060243756874748}


In [ ]:
result

{'eval_loss': 9.060243756874748, 'mcc': -0.010226134499304049}

In [ ]:
predictions, raw_outputs = model.predict(["Obama is the best in the world."])

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


In [ ]:
predictions[1]

IndexError: ignored

In [ ]:
!cp /content/outputs/pytorch_model.bin "/content/drive/My Drive/Machine Learning/simpleTransformers-News"

In [ ]:
!cp /content/outputs/*.json "/content/drive/My Drive/Machine Learning/simpleTransformers-News"

In [ ]:
!cp /content/outputs/*.txt "/content/drive/My Drive/Machine Learning/simpleTransformers-News"

In [ ]:
!cp /content/outputs/training_args.bin "/content/drive/My Drive/Machine Learning/simpleTransformers-News"

# Loading and Testing

In [17]:
model = ClassificationModel('bert', "/content/drive/My Drive/Machine Learning/simpleTransformers-News", num_labels=41, use_cuda=True, 
                            args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs':10, 'train_batch_size':128})

In [ ]:
predictions, raw_outputs=model.predict(["Singapore’s election: Why aren’t the winners smiling?"])

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


In [ ]:
predictions

[40]

# Eval

In [18]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:810: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_model:{'mcc': 0.8346433224348765, 'eval_loss': 0.7504636307434239}


In [24]:
len(test)

40172

In [25]:
len(wrong_predictions)/len(test)

0.15575525241461716

In [20]:
result

{'eval_loss': 0.7504636307434239, 'mcc': 0.8346433224348765}

In [22]:
wrong_predictions[0].__dict__

{'guid': 4,
 'label': 22,
 'text_a': 'Waiting One More Day (on DOMA)',
 'text_b': None}

In [23]:
model.predict(["Waiting One More Day (on DOMA)"])

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


([25], array([[ 5.3   ,  0.814 , -1.812 , -0.3176, -1.142 ,  0.9507, -3.645 ,
          4.26  , -1.135 , -0.5767,  2.314 , -1.59  , -0.2615, -0.63  ,
         -0.3525, -0.768 , -0.914 ,  1.097 , -0.1488, -2.143 , -2.467 ,
         -2.238 ,  0.7896, -0.2556, -1.284 ,  7.17  ,  0.511 , -0.8184,
         -1.301 , -0.6577, -0.1355, -1.425 ,  1.412 , -1.02  ,  2.326 ,
         -0.5327, -0.2349,  1.142 , -1.106 , -3.041 ,  0.3506]],
       dtype=float16))